# Text Embeddings Generation - Par Source

**Objectif**: Générer embeddings textuels (CamemBERT 768 dim) de manière séparée par source

**Méthodologie**:
1. **IA_only**: Embeddings pour les 73 produits Données_IA (source IID)
2. **Feedtable_only**: Embeddings pour les 122 produits Feedtable (source OOD, excluant duplicatas IA)
3. **Combined**: Embeddings pour tous les produits (union des deux sources)

**Prérequis**: 
- `data_merged_with_ood_classification.xlsx` (données intégrées)
- Accès aux sources: `source` colonne indiquant 'Donnees IA' ou 'Feedtable'

In [15]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = '../data/'
MODEL = 'dangvantuan/sentence-camembert-base'
print(" Configuration chargée")

 Configuration chargée


## 1. Charger données + identifier sources

Charger les données intégrées, identifier les noms uniques par source (Données IA vs Feedtable), préparer les textes combinés.

In [16]:
# Charger données
df = pd.read_excel(DATA_DIR + 'data_merged_with_ood_classification.xlsx')
print(f"✓ Chargé: {len(df)} lignes")

# Vérifier colonnes requises
required_cols = ['Nom', 'Definition', 'source', 'OOD']
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"Colonnes requises manquantes: {required_cols}")

# Préparer textes (Nom + Definition)
df['text'] = df['Nom'].astype(str) + "; " + df['Definition'].astype(str)

# ===== ÉTAPE 1: Données IA (IID - source principale, OOD=0) =====
df_ia = df[df['source'] == 'Donnees_IA'].copy()
noms_ia = set(df_ia['Nom'].unique())

print(f"\n Données IA (source='Donnees_IA', OOD=0):")
print(f"  - Lignes: {len(df_ia)}")
print(f"  - Noms uniques: {len(noms_ia)}")

# Textes uniques pour IA
texts_ia_unique = df_ia['text'].drop_duplicates().tolist()
print(f"  - Textes uniques: {len(texts_ia_unique)}")

# ===== ÉTAPE 2: Feedtable (OOD pure - source secondaire, OOD=1) =====
# Filtrer UNIQUEMENT les Feedtables vraiment nouveaux (OOD=1 selon annotations manuelles)
df_feedtable_ood = df[(df['source'] == 'Feedtable') & (df['OOD'] == 1)].copy()

print(f"\n Feedtable OOD-only (source='Feedtable', OOD=1 - basé sur annotations):")
print(f"  - Lignes totales source Feedtable: {len(df[df['source'] == 'Feedtable'])}")
print(f"  - Lignes OOD=0 (même que IA): {len(df[(df['source'] == 'Feedtable') & (df['OOD'] == 0)])}")
print(f"  - Lignes OOD=1 (vrais nouveaux): {len(df_feedtable_ood)}")
print(f"  - Noms uniques vrais OOD: {df_feedtable_ood['Nom'].nunique()}")

texts_feedtable_ood_unique = df_feedtable_ood['text'].drop_duplicates().tolist()
print(f"  - Textes uniques: {len(texts_feedtable_ood_unique)}")

# ===== ÉTAPE 3: Combined (tous les produits) =====
texts_combined_unique = df['text'].drop_duplicates().tolist()
print(f"\n Combined (tous les produits):")
print(f"  - Lignes totales: {len(df)}")
print(f"  - Noms uniques: {len(df['Nom'].unique())}")
print(f"  - Textes uniques: {len(texts_combined_unique)}")
print(f"  - IID (source IA + Feedtable OOD=0): {len(df[df['OOD'] == 0])}")
print(f"  - OOD (Feedtable OOD=1): {len(df[df['OOD'] == 1])}")

✓ Chargé: 6550 lignes

 Données IA (source='Donnees_IA', OOD=0):
  - Lignes: 6352
  - Noms uniques: 73
  - Textes uniques: 73

 Feedtable OOD-only (source='Feedtable', OOD=1 - basé sur annotations):
  - Lignes totales source Feedtable: 198
  - Lignes OOD=0 (même que IA): 76
  - Lignes OOD=1 (vrais nouveaux): 122
  - Noms uniques vrais OOD: 122
  - Textes uniques: 122

 Combined (tous les produits):
  - Lignes totales: 6550
  - Noms uniques: 261
  - Textes uniques: 261
  - IID (source IA + Feedtable OOD=0): 6428
  - OOD (Feedtable OOD=1): 122


## 2. Générer embeddings (CamemBERT 768 dim) par source

Charger le modèle une seule fois, générer les embeddings pour IA, Feedtable (new), et Combined.

In [17]:
# Charger modèle
print(f"Chargement {MODEL}...")
model = SentenceTransformer(MODEL)

# ===== Fonction helper pour générer embeddings =====
def generate_embeddings_for_source(texts_unique, df_source, data_label):
    """Générer embeddings unique et tous les embeddings pour une source"""
    print(f"\n{'='*60}")
    print(f" Genération embeddings pour {data_label}")
    print(f"{'='*60}")
    
    # Embeddings textes uniques
    print(f"Génération embeddings {len(texts_unique)} textes uniques...")
    emb_unique = model.encode(texts_unique, show_progress_bar=True, batch_size=32)
    
    # Mapping pour tous les produits
    text_to_idx = {text: idx for idx, text in enumerate(texts_unique)}
    indices = df_source['text'].map(text_to_idx).values
    emb_all = emb_unique[indices]
    
    print(f" Embeddings générés:")
    print(f"  - Unique: {emb_unique.shape} ({emb_unique.nbytes/1e6:.1f} MB)")
    print(f"  - Tous: {emb_all.shape} ({emb_all.nbytes/1e6:.1f} MB)")
    print(f"  - Noms uniques: {df_source['Nom'].nunique()}")
    
    return emb_unique, emb_all, indices

# ===== Générer embeddings par source =====

# 1. IA_only
emb_ia_unique, emb_ia_all, idx_ia = generate_embeddings_for_source(
    texts_ia_unique, df_ia, "IA_only (source='Donnees_IA')"
)

# 2. Feedtable_only (vrais OOD uniquement)
if len(df_feedtable_ood) > 0:
    emb_ft_unique, emb_ft_all, idx_ft = generate_embeddings_for_source(
        texts_feedtable_ood_unique, df_feedtable_ood, "Feedtable_only (source='Feedtable', OOD=1)"
    )
else:
    emb_ft_unique, emb_ft_all, idx_ft = None, None, None
    print(f"\n  Pas de Feedtable OOD=1, génération skippée")

# 3. Combined
emb_combined_unique, emb_combined_all, idx_combined = generate_embeddings_for_source(
    texts_combined_unique, df, "Combined (tous les produits)"
)

print(f"\n{'='*60}")
print(f" Embeddings générés pour les 3 versions")
print(f"{'='*60}")

Chargement dangvantuan/sentence-camembert-base...

 Genération embeddings pour IA_only (source='Donnees_IA')
Génération embeddings 73 textes uniques...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

 Embeddings générés:
  - Unique: (73, 768) (0.2 MB)
  - Tous: (6352, 768) (19.5 MB)
  - Noms uniques: 73

 Genération embeddings pour Feedtable_only (source='Feedtable', OOD=1)
Génération embeddings 122 textes uniques...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

 Embeddings générés:
  - Unique: (122, 768) (0.4 MB)
  - Tous: (122, 768) (0.4 MB)
  - Noms uniques: 122

 Genération embeddings pour Combined (tous les produits)
Génération embeddings 261 textes uniques...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

 Embeddings générés:
  - Unique: (261, 768) (0.8 MB)
  - Tous: (6550, 768) (20.1 MB)
  - Noms uniques: 261

 Embeddings générés pour les 3 versions


## 3. Export final - Sauvegarder les 3 versions

Pour chaque source (IA, Feedtable_new, Combined):
- Embeddings uniques (.npy)
- Tous les embeddings (.npy)
- Dataframe enrichi avec indices (.xlsx)

In [18]:
# ===== Sauvegarder les 3 versions =====

def save_embeddings_version(emb_unique, emb_all, df_source, idx_array, suffix):
    """Sauvegarder embeddings et dataframe enrichi pour une version"""
    
    # IMPORTANT: Réassigner les embedding_idx de manière séquentielle
    # Cela garantit que les embeddings correspondent directement à l'ordre du dataframe
    df_save = df_source.copy()
    
    # Créer un mapping Nom -> embedding pour chaque source
    # car les embeddings sont générés par texte unique, pas par ligne
    from sklearn.preprocessing import LabelEncoder
    
    # Réassigner des indices simples 0, 1, 2, ..., len(df_save)-1
    # Ces indices correspondent DIRECTEMENT aux positions dans emb_all
    df_save['embedding_idx'] = np.arange(len(df_save))
    
    filename_unique = f'embeddings_{suffix}_unique.npy'
    filename_all = f'embeddings_{suffix}_all.npy'
    filename_xlsx = f'data_{suffix}_with_embeddings_index.xlsx'
    
    # IMPORTANT: Vérifier l'alignement
    assert len(df_save) == emb_all.shape[0], f"Mismatch: {len(df_save)} rows vs {emb_all.shape[0]} embeddings"
    
    # Sauvegarder embeddings
    np.save(DATA_DIR + filename_unique, emb_unique)
    np.save(DATA_DIR + filename_all, emb_all)
    
    # Sauvegarder dataframe
    df_save.to_excel(DATA_DIR + filename_xlsx, index=False)
    
    print(f"\n ✓ {suffix}:")
    print(f"  - {filename_unique} ({emb_unique.nbytes/1e6:.1f} MB, {emb_unique.shape[0]} unique)")
    print(f"  - {filename_all} ({emb_all.nbytes/1e6:.1f} MB, {emb_all.shape[0]} all)")
    print(f"  - {filename_xlsx} ({len(df_save)} lignes)")
    print(f"   Alignment verified: embedding_idx range [0, {len(df_save)-1}]")
    
    return filename_unique, filename_all, filename_xlsx

# 1. IA_only
file_ia_unique, file_ia_all, file_ia_xlsx = save_embeddings_version(
    emb_ia_unique, emb_ia_all, df_ia, idx_ia, "IA_only"
)

# 2. Feedtable_only (vrais OOD uniquement)
if emb_ft_unique is not None:
    file_ft_unique, file_ft_all, file_ft_xlsx = save_embeddings_version(
        emb_ft_unique, emb_ft_all, df_feedtable_ood, idx_ft, "Feedtable_only"
    )

# 3. Combined
file_combined_unique, file_combined_all, file_combined_xlsx = save_embeddings_version(
    emb_combined_unique, emb_combined_all, df, idx_combined, "combined"
)

print(f"\n{'='*60}")
print(f"\n{'='*60}")
print(f" EXPORT FINALISÉ")
print(f"{'='*60}")
print(f"\n Résumé des fichiers générés:")
print(f"\n  IA_only (source='Donnees_IA'):")
print(f"    - embeddings_IA_only_unique.npy")
print(f"    - embeddings_IA_only_all.npy")
print(f"    - data_IA_only_with_embeddings_index.xlsx")

if emb_ft_unique is not None:
    print(f"\n  Feedtable_only (source='Feedtable', OOD=1 selon annotations):")
    print(f"    - embeddings_Feedtable_only_unique.npy")
    print(f"    - embeddings_Feedtable_only_all.npy")
    print(f"    - data_Feedtable_only_with_embeddings_index.xlsx")

print(f"\n  Combined (tous les produits):")
print(f"    - embeddings_combined_unique.npy")
print(f"    - embeddings_combined_all.npy")
print(f"    - data_combined_with_embeddings_index.xlsx")


 ✓ IA_only:
  - embeddings_IA_only_unique.npy (0.2 MB, 73 unique)
  - embeddings_IA_only_all.npy (19.5 MB, 6352 all)
  - data_IA_only_with_embeddings_index.xlsx (6352 lignes)
   Alignment verified: embedding_idx range [0, 6351]

 ✓ Feedtable_only:
  - embeddings_Feedtable_only_unique.npy (0.4 MB, 122 unique)
  - embeddings_Feedtable_only_all.npy (0.4 MB, 122 all)
  - data_Feedtable_only_with_embeddings_index.xlsx (122 lignes)
   Alignment verified: embedding_idx range [0, 121]

 ✓ combined:
  - embeddings_combined_unique.npy (0.8 MB, 261 unique)
  - embeddings_combined_all.npy (20.1 MB, 6550 all)
  - data_combined_with_embeddings_index.xlsx (6550 lignes)
   Alignment verified: embedding_idx range [0, 6549]


 EXPORT FINALISÉ

 Résumé des fichiers générés:

  IA_only (source='Donnees_IA'):
    - embeddings_IA_only_unique.npy
    - embeddings_IA_only_all.npy
    - data_IA_only_with_embeddings_index.xlsx

  Feedtable_only (source='Feedtable', OOD=1 selon annotations):
    - embeddings_Fe

## 4. Analyse comparative des trois versions

Visualiser et comparer les embeddings selon les sources : IA_only, Feedtable_only, Combined

In [19]:
import sys
sys.path.insert(0, '../')

from src.visu import plot_embeddings_3d_pca

# ===== Fonction helper pour visualiser =====
def visualize_embeddings(embeddings, df_with_emb, version_label):
    """Visualiser embeddings par OOD et par Nom"""
    print(f"\n{'='*60}")
    print(f" Visualisation {version_label}")
    print(f"{'='*60}")
    
    # Par OOD
    if 'OOD' in df_with_emb.columns:
        fig_ood, pca = plot_embeddings_3d_pca(
            embeddings, 
            df_with_emb, 
            hue_col='OOD',
            title=f'PCA 3D - {version_label} (par OOD)'
        )
        fig_ood.show()
        print(f" Visualisation par OOD:")
        print(f"  - IID (Bleu): {(df_with_emb['OOD']==0).sum()} échantillons")
        print(f"  - OOD (Rouge): {(df_with_emb['OOD']==1).sum()} échantillons")
        print(f"  - Variance expliquée (PC1+PC2+PC3): {pca.explained_variance_ratio_[:3].sum():.1%}")
    
    # Par Nom
    fig_nom, _ = plot_embeddings_3d_pca(
        embeddings, 
        df_with_emb, 
        hue_col='Nom', 
        title=f'PCA 3D - {version_label} (par Nom)'
    )
    fig_nom.show()
    print(f" Visualisation par Nom ({df_with_emb['Nom'].nunique()} produits)")

# ===== Charger et visualiser les 3 versions =====

# IA_only
print("\n IA_only (source='Donnees_IA')")
emb_ia = np.load(DATA_DIR + 'embeddings_IA_only_all.npy')
df_ia_emb = pd.read_excel(DATA_DIR + 'data_IA_only_with_embeddings_index.xlsx')
visualize_embeddings(emb_ia, df_ia_emb, "IA_only")

# Feedtable_only
if emb_ft_unique is not None:
    print("\n Feedtable_only (source='Feedtable', OOD=1)")
    emb_ft = np.load(DATA_DIR + 'embeddings_Feedtable_only_all.npy')
    df_ft_emb = pd.read_excel(DATA_DIR + 'data_Feedtable_only_with_embeddings_index.xlsx')
    visualize_embeddings(emb_ft, df_ft_emb, "Feedtable_only")

# Combined
print("\n Combined (tous les produits)")
emb_combined = np.load(DATA_DIR + 'embeddings_combined_all.npy')
df_combined_emb = pd.read_excel(DATA_DIR + 'data_combined_with_embeddings_index.xlsx')
visualize_embeddings(emb_combined, df_combined_emb, "Combined")


 IA_only (source='Donnees_IA')

 Visualisation IA_only


 Visualisation par OOD:
  - IID (Bleu): 6352 échantillons
  - OOD (Rouge): 0 échantillons
  - Variance expliquée (PC1+PC2+PC3): 27.7%


 Visualisation par Nom (73 produits)

 Feedtable_only (source='Feedtable', OOD=1)

 Visualisation Feedtable_only


 Visualisation par OOD:
  - IID (Bleu): 0 échantillons
  - OOD (Rouge): 122 échantillons
  - Variance expliquée (PC1+PC2+PC3): 24.1%


 Visualisation par Nom (122 produits)

 Combined (tous les produits)

 Visualisation Combined


 Visualisation par OOD:
  - IID (Bleu): 6428 échantillons
  - OOD (Rouge): 122 échantillons
  - Variance expliquée (PC1+PC2+PC3): 27.5%


 Visualisation par Nom (261 produits)
